<a href="https://colab.research.google.com/github/abdyraman/credit-risk-classification/blob/main/credit_risk_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import sklearn as skl

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [48]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
lending_df=pd.read_csv('/content/lending_data.csv')

# Review the DataFrame
lending_df.sample(4)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
62426,11800.0,8.131,57100,0.474606,6,1,27100,0
17338,10700.0,7.673,52800,0.431818,5,1,22800,0
58365,10400.0,7.559,51700,0.419729,4,1,21700,0
33910,8600.0,6.759,44200,0.321267,3,0,14200,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [49]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns=['loan_status'])

In [50]:
# Review the y variable Series
y.shape

(77536,)

In [51]:
# Review the X variable DataFrame
X.shape

(77536, 7)

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [71]:
# Check the balance of our target values
value_counts = y.value_counts()
print(value_counts)

0    75036
1     2500
Name: loan_status, dtype: int64


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [53]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [54]:
# For better modeling I created scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

StandardScaler()

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [55]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

In [56]:
# Checking the training y values
value_counts = y_train.value_counts()
print(value_counts)

0    56253
1     1899
Name: loan_status, dtype: int64


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [57]:
# Make a prediction using the testing data
testing_predictions = logistic_regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [58]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score
# Instantiate the logistic regression classifier
log_classifier = LogisticRegression()

# Fit the model to your training data
log_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = log_classifier.predict(X_test)

# Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic regression model accuracy: {accuracy:.3f}")

Logistic regression model accuracy: 0.992


In [59]:
# Generate a confusion matrix for the model
test_matrix = confusion_matrix(y_test, testing_predictions)
# Print the confusion matrix for the testing data
print(test_matrix)

[[18678   105]
 [   55   546]]


In [60]:
# Print the classification report for the model
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18783
           1       0.84      0.91      0.87       601

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.93     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** This model demonstrates higher accuracy in predicting healthy loans compared to high-risk loans. The precision for predicting high-risk loans is reported to be 84%, with an F1 score of 87%. Conversely, for positive loans, the precision and F1 score are both 100%.

The confusion matrix indicates that out of 546 sample points, there are 55 false negatives. Its a high number.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [61]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)
# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train, y_train)

In [62]:
# Count the distinct values of the resampled labels data

import numpy as np

# Count the distinct values of the resampled labels data
unique_labels = np.unique(y_train_resampled)
num_classes = len(unique_labels)

print("Number of distinct classes:", num_classes)
print("Distinct classes:", unique_labels)


Number of distinct classes: 2
Distinct classes: [0 1]


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [63]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model_ros = LogisticRegression(random_state=1)
# Fit the model using the resampled training data
logistic_regression_model_ros.fit(X_train_resampled, y_train_resampled)
# Make a prediction using the testing data
y_pred_ros = logistic_regression_model_ros.predict(X_test)

In [64]:
# Checking the resampled training y values
value_counts = y_train_resampled.value_counts()
print(value_counts)

0    56253
1    56253
Name: loan_status, dtype: int64


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [70]:
# Print the balanced_accuracy score of the model
balanced_accuracy = balanced_accuracy_score(y_test, y_pred_ros)
formatted_accuracy = "{:.3f}".format(balanced_accuracy)
print("Balanced accuracy score:", formatted_accuracy)

Balanced accuracy score: 0.994


In [24]:
# Generate a confusion matrix for the model
confusion_mtx = confusion_matrix(y_test, y_pred_ros)
print("Confusion Matrix:")
print(confusion_mtx)

Confusion Matrix:
[[18646    90]
 [    3   645]]


In [25]:
# Print the classification report for the model
classification_rep = classification_report(y_test, y_pred_ros)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18736
           1       0.88      1.00      0.93       648

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.97     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The improved model, achieved through the utilization of random oversampling, demonstrates enhanced performance compared to the previous model. This enhancement is attributed to the equalization of the sample count for the class labeled as 1, now matching that of the class labeled as 0, resulting in 56,253 samples for each class.

As a consequence of this adjustment, the number of false negatives in the confusion matrix decreased significantly to just 3. However, it is worth noting that the false positives increased compared to the previous model.

Additionally, the classification report reveals a higher recall of 100% for the class labeled as 0, compared to the previous model's recall of 99%. Furthermore, this improved model exhibits a slight increase in precision and F1 score for the class labeled as 0.

The overall accuracy of the model remained nearly the same, maintaining a high level of 99%, which aligns with the accuracy achieved by the previous model.